In [1]:
!pip install keras-vggface

In [1]:
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Conv2D, Input, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
import random
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.constraints import Constraint
import math

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = 224
vggface = VGGFace(model='vgg16', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

In [3]:
vggface.layers.pop()
vggface.layers.pop()
layers = [l for l in vggface.layers]
for i in range(0, 18):
     layers[i].trainable = False

x = Dropout(0.7, name = 'Dropout')(layers[21].output)
for i in range(22, len(layers)):
    x = layers[i](x)

x = Dense(7, activation = 'softmax')(x)    
model = Model(inputs = layers[0].input, outputs = x)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0   

In [4]:
train_folder = "C:/Users/nico9/Desktop/FaceDetection/Train"
val_folder = "C:/Users/nico9/Desktop/FaceDetection/Val"

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True)

In [5]:
train_batchsize = 32
val_batchsize = 8

train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=train_batchsize,
        class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
        val_folder,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=val_batchsize,
        class_mode="categorical",
        shuffle=False
)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [6]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1 , patience = 10)

callbacks_list = [reduce_lr]

In [7]:
num_epochs = 200
learning_rate = 1e-4
sgd = SGD(lr=learning_rate, momentum = 0.9)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['acc'])

In [8]:
history = model.fit_generator(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1,
    callbacks=callbacks_list
)

model.save('../saves/vggFace')

Epoch 1/200
898/898 [==============================] - 261s 291ms/step - loss: 1.8089 - acc: 0.4421 - val_loss: 1.7915 - val_acc: 0.5300
Epoch 2/200
898/898 [==============================] - 256s 285ms/step - loss: 1.2680 - acc: 0.5426 - val_loss: 1.3672 - val_acc: 0.5620
Epoch 3/200
898/898 [==============================] - 256s 285ms/step - loss: 1.1419 - acc: 0.5837 - val_loss: 1.4073 - val_acc: 0.5865
Epoch 4/200
898/898 [==============================] - 256s 285ms/step - loss: 1.0380 - acc: 0.6182 - val_loss: 1.4830 - val_acc: 0.6002
Epoch 5/200
898/898 [==============================] - 256s 285ms/step - loss: 0.9708 - acc: 0.6462 - val_loss: 1.3208 - val_acc: 0.6066
Epoch 6/200
898/898 [==============================] - 255s 285ms/step - loss: 0.9058 - acc: 0.6691 - val_loss: 1.4162 - val_acc: 0.6194
Epoch 7/200
898/898 [==============================] - 255s 285ms/step - loss: 0.8498 - acc: 0.6874 - val_loss: 1.5796 - val_acc: 0.6247
Epoch 8/200
898/898 [====================

898/898 [==============================] - 256s 285ms/step - loss: 0.3196 - acc: 0.8921 - val_loss: 1.8776 - val_acc: 0.6698
Epoch 61/200
898/898 [==============================] - 256s 285ms/step - loss: 0.3216 - acc: 0.8904 - val_loss: 1.5878 - val_acc: 0.6659
Epoch 62/200
898/898 [==============================] - 256s 285ms/step - loss: 0.3179 - acc: 0.8922 - val_loss: 1.5892 - val_acc: 0.6651
Epoch 63/200
898/898 [==============================] - 256s 285ms/step - loss: 0.3168 - acc: 0.8924 - val_loss: 1.9074 - val_acc: 0.6721
Epoch 64/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3201 - acc: 0.8922 - val_loss: 1.9068 - val_acc: 0.6726
Epoch 65/200
898/898 [==============================] - 255s 285ms/step - loss: 0.3225 - acc: 0.8910 - val_loss: 1.8775 - val_acc: 0.6668
Epoch 66/200
898/898 [==============================] - 256s 285ms/step - loss: 0.3214 - acc: 0.8874 - val_loss: 1.5878 - val_acc: 0.6687
Epoch 67/200
898/898 [=========================

898/898 [==============================] - 255s 284ms/step - loss: 0.3216 - acc: 0.8920 - val_loss: 1.5918 - val_acc: 0.6707
Epoch 120/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3166 - acc: 0.8931 - val_loss: 1.8755 - val_acc: 0.6712
Epoch 121/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3225 - acc: 0.8889 - val_loss: 1.8755 - val_acc: 0.6707
Epoch 122/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3197 - acc: 0.8918 - val_loss: 1.6197 - val_acc: 0.6665
Epoch 123/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3198 - acc: 0.8890 - val_loss: 1.8749 - val_acc: 0.6679
Epoch 124/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3179 - acc: 0.8920 - val_loss: 1.8749 - val_acc: 0.6662
Epoch 125/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3184 - acc: 0.8915 - val_loss: 1.6177 - val_acc: 0.6704
Epoch 126/200
898/898 [==================

898/898 [==============================] - 255s 284ms/step - loss: 0.3180 - acc: 0.8922 - val_loss: 1.8755 - val_acc: 0.6668
Epoch 179/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3209 - acc: 0.8913 - val_loss: 1.9068 - val_acc: 0.6709
Epoch 180/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3190 - acc: 0.8894 - val_loss: 1.5858 - val_acc: 0.6668
Epoch 181/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3201 - acc: 0.8909 - val_loss: 1.6177 - val_acc: 0.6676
Epoch 182/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3201 - acc: 0.8917 - val_loss: 1.9108 - val_acc: 0.6668
Epoch 183/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3209 - acc: 0.8916 - val_loss: 1.8803 - val_acc: 0.6665
Epoch 184/200
898/898 [==============================] - 255s 284ms/step - loss: 0.3135 - acc: 0.8942 - val_loss: 1.6217 - val_acc: 0.6695
Epoch 185/200
898/898 [==================

# Fase Sparse

In [ ]:
class Sparse(Constraint):
    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)
    
    def __call__(self,x):
        return self.mask * x
    
    def get_config(self):
        return {'mask': self.mask}

In [ ]:
def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

In [ ]:
masks = create_sparsity_masks(model,30)

In [ ]:
sparse_model = Sequential([
    Conv2D(32,3,input_shape=(28,28,1), kernel_constraint=Sparse(masks[0])),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,3, kernel_constraint=Sparse(masks[1])),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(250, kernel_constraint=Sparse(masks[2])),
    Activation('relu'),
    Dropout(0.4),
    Dense(10, kernel_constraint=Sparse(masks[3])),
    Activation('softmax')
])

adam = Adam()
sparse_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
sparse_model.summary()
#Get weights from densely trained model
sparse_model.set_weights(model.get_weights())

In [ ]:
num_epochs = 50
history = sparse_model.fit_generator(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1,
    callbacks=callbacks_list
)

# Fase Dense

In [ ]:
redense_model = Sequential([
    Conv2D(32,3,input_shape=(28,28,1)),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,3),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(250),
    Activation('relu'),
    Dropout(0.4),
    Dense(10),
    Activation('softmax')
])

adam = Adam(lr=0.0001)#Default Adam lr is 0.001 so I set it to 0.0001
redense_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
redense_model.summary()
#Get weights from sparsely trained model
redense_model.set_weights(sparse_model.get_weights())

In [ ]:
num_epochs = 50
history = redense_model.fit_generator(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1,
    callbacks=callbacks_list
)